In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = 'poems.txt'

In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 9964421 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

Never Enough of Living
Never, my heart, is there enough of living,
Since only in thee is loveliness so sweet pain;
Only for thee the willows will be giving
Their quiet fringes to the dreaming river;
Only for thee so the light grasses ever
Are h


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

159 unique characters


Now create the tf.keras.layers.StringLookup layer:

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task
Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

### Create training examples and targets
Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [10]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(9964421,), dtype=int64, numpy=array([50, 73, 90, ...,  2,  3,  2], dtype=int64)>

In [11]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [12]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

N
e
v
e
r
 
E
n
o
u


In [13]:
seq_length = 100

In [14]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'N' b'e' b'v' b'e' b'r' b' ' b'E' b'n' b'o' b'u' b'g' b'h' b' ' b'o'
 b'f' b' ' b'L' b'i' b'v' b'i' b'n' b'g' b'\r' b'\n' b'N' b'e' b'v' b'e'
 b'r' b',' b' ' b'm' b'y' b' ' b'h' b'e' b'a' b'r' b't' b',' b' ' b'i'
 b's' b' ' b't' b'h' b'e' b'r' b'e' b' ' b'e' b'n' b'o' b'u' b'g' b'h'
 b' ' b'o' b'f' b' ' b'l' b'i' b'v' b'i' b'n' b'g' b',' b'\r' b'\n' b'S'
 b'i' b'n' b'c' b'e' b' ' b'o' b'n' b'l' b'y' b' ' b'i' b'n' b' ' b't'
 b'h' b'e' b'e' b' ' b'i' b's' b' ' b'l' b'o' b'v' b'e' b'l' b'i' b'n'
 b'e' b's' b's'], shape=(101,), dtype=string)


In [15]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'Never Enough of Living\r\nNever, my heart, is there enough of living,\r\nSince only in thee is loveliness'
b' so sweet pain;\r\nOnly for thee the willows will be giving\r\nTheir quiet fringes to the dreaming river;'
b'\r\nOnly for thee so the light grasses ever\r\nAre hollowed by the print of windy feet,\r\nAnd breathe hill'
b' weather on the misty plain;\r\nAnd were no rapture of them in thy beat,\r\nFor every hour of sky\r\nStillb'
b'orn in gladness would the waters wear\r\nColors of air translucently,\r\nAnd the stars sleep there.\r\nGent'


In [16]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [17]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [18]:
dataset = sequences.map(split_input_target)

In [19]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Never Enough of Living\r\nNever, my heart, is there enough of living,\r\nSince only in thee is lovelines'
Target: b'ever Enough of Living\r\nNever, my heart, is there enough of living,\r\nSince only in thee is loveliness'


In [20]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [21]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [23]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [24]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [25]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 160) # (batch_size, sequence_length, vocab_size)


In [26]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  40960     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  164000    
                                                                 
Total params: 4,143,264
Trainable params: 4,143,264
Non-trainable params: 0
_________________________________________________________________


In [27]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [28]:
sampled_indices

array([ 77,  58, 137,  18,  37, 126, 136, 113, 104, 129,  99,  36,  83,
        45,  88,  14,  78,   3,  35,  40, 107,  51,  14,  91,  48,  82,
       133,  92, 105, 114,  75,  88,  86,  12, 151, 128, 136, 123,  22,
        98,  78,  49, 117,  34,   8,  29,  31, 121, 109,  68,  14,  19,
        69,  51,   6,  20, 109, 127,  42, 101, 109, 149,  81, 151,  74,
        87,  94, 112,  33, 123,  73, 132,  87, 120,  67, 116,  92,  24,
         1,  16,  33,  41,   6, 100,   0, 114, 118,  57,  97,  83,  76,
       150,  79,  75,  25, 141,  94, 102,  45, 121], dtype=int64)

In [29]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'he\r\ndetails of death\xe2\x80\x99s arrival at a defenseless moment.\r\nCan you hear the snow as it settles its dri'

Next Char Predictions:
 b'iV\xc3\xb4.A\xc3\xa6\xc3\xb3\xc2\xbb\xc2\xab\xc3\xa9\xc2\xa0@oIt*j\r?D\xc2\xb0O*wLn\xc3\xaex\xc2\xad\xc3\x80gtr(\xe2\x80\x9d\xc3\xa8\xc3\xb3\xc3\xa22~jM\xc3\x86>$9;\xc3\xa0\xc2\xb4`*/aO"0\xc2\xb4\xc3\xa7F\xc2\xa2\xc2\xb4\xe2\x80\x99m\xe2\x80\x9dfsz\xc2\xb9=\xc3\xa2e\xc3\xads\xc3\x89_\xc3\x85x4\t,=E"\xc2\xa1[UNK]\xc3\x80\xc3\x87U}oh\xe2\x80\x9ckg5\xc3\xbbz\xc2\xa6I\xc3\xa0'


In [30]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [31]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 160)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(5.075081, shape=(), dtype=float32)


In [32]:
tf.exp(example_batch_mean_loss).numpy()

159.98512

In [33]:
model.compile(optimizer='adam', loss=loss)

In [34]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [40]:
EPOCHS = 25

In [41]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback]) 

Epoch 1/25
1541/1541 [==============================] - 3656s 2s/step - loss: 1.1346
Epoch 2/25
1541/1541 [==============================] - 3881s 3s/step - loss: 1.1253
Epoch 3/25
1541/1541 [==============================] - 3896s 3s/step - loss: 1.1345
Epoch 4/25
1541/1541 [==============================] - 3893s 3s/step - loss: 1.1222
Epoch 5/25
1541/1541 [==============================] - 3905s 3s/step - loss: 1.1226
Epoch 6/25
1541/1541 [==============================] - 3886s 3s/step - loss: 1.1338
Epoch 7/25
1541/1541 [==============================] - 3892s 3s/step - loss: 1.2266
Epoch 8/25
1541/1541 [==============================] - 3891s 3s/step - loss: 1.3447
Epoch 9/25
1541/1541 [==============================] - 3900s 3s/step - loss: 1.3464
Epoch 10/25
1541/1541 [==============================] - 3979s 3s/step - loss: 1.2050
Epoch 11/25
1541/1541 [==============================] - 4218s 3s/step - loss: 1.2365
Epoch 12/25
1541/1541 [==============================] - 4314s 

In [42]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [43]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

start = time.time()
states = None
next_char = tf.constant(['Humpty dumpty'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

In [46]:
start = time.time()
states = None
next_char = tf.constant(['Happy valentine'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Happy valentine-slave’s stones on the confused,
me, my death your re’d need in unjustion
Again,   
But you’re here his clay for suckets,
Recalling the rain burning sleep-start.
But we’ve ladies. 

My Fretowin her de through the Garden.
So stry clean in bore it spell.
I move Over a short cance ignoinias their born by tears far afterman passions and in the road
The last before the stretchislection.
Now their fair, agian sadness,
With my people play recall
To follows Blind...
A all weak in said it all of other.
In a morning,
than that my superstition.
Say that moment you not dalky,
Dead us from tended...
Like make head pains, like eet gife-his heart – but in enter,
a net you’ll feel:
we have like a way out to other.

The Returning of Love
Confining friend.
Millier in your burning of far to remember or tears,
A reflections hot windows, pent, ratter,
Then a boxt to right my life soul, not enly noise singer
the snow, my decide
lost immebird spealt pines of people bard.
And I’m 

____________